In [1]:
import pandas as pd

In [ ]:
# Read data january of 2023
data_jan = pd.read_parquet('./data/homework_1/yellow_tripdata_2023-01.parquet')

In [7]:
# Task 1: What's the standard deviation of the trips duration in January?
len(data_jan.columns)

19

In [8]:
# Task 2: Now let's compute the duration variable. It should contain the duration of a ride in minutes.
# What's the standard deviation of the trips duration in January?

# Compute duration in minutes
data_jan['duration'] = (data_jan['tpep_dropoff_datetime'] - data_jan['tpep_pickup_datetime']).dt.total_seconds() / 60

# Calculate the standard deviation of the duration
std_duration = data_jan['duration'].std()
std_duration

np.float64(42.59435124195458)

In [9]:
# Task 3: Next, we need to check the distribution of the duration variable. 
# There are some outliers. 
# Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

# What fraction of the records left after you dropped the outliers?


# Filter the data to keep only durations between 1 and 60 minutes (inclusive)
filtered_data = data_jan[(data_jan['duration'] >= 1) & (data_jan['duration'] <= 60)]

# Calculate the fraction of records left
fraction_left = len(filtered_data) / len(data_jan)
fraction_left

0.9812202822125979

In [10]:
# Task 4: Let's apply one-hot encoding to the pickup and dropoff location IDs. 
# We'll use only these two features for our model.

# - Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
# - Fit a dictionary vectorizer
# - Get a feature matrix from it
# What's the dimensionality of this matrix (number of columns)? 


from sklearn.feature_extraction import DictVectorizer

# Select only the relevant columns and re-cast the IDs to strings
categorical = filtered_data[['PULocationID', 'DOLocationID']].astype(str)

# Turn the DataFrame into a list of dictionaries
dicts = categorical.to_dict(orient='records')

# Fit a dictionary vectorizer
dv = DictVectorizer()
X_train = dv.fit_transform(dicts)

# Get the dimensionality (number of columns)
X_train.shape[1]

515

In [11]:
# Task 5: Now let's use the feature matrix from the previous step to train a model.

# Train a plain linear regression model with default parameters, where duration is the response variable
# Calculate the RMSE of the model on the training data
# What's the RMSE on train?


from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Prepare the target variable
y_train = filtered_data['duration'].values

# Train the linear regression model
lr = LinearRegression()
lr.fit(X_train, y_train)

# Predict on the training data
y_pred = lr.predict(X_train)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_train, y_pred))
rmse

np.float64(7.649261946248764)

In [12]:
# Task 6: Now let's apply this model to the validation dataset (February 2023).

# What's the RMSE on validation?

# 1. Read the February data
data_feb = pd.read_parquet('./data/homework_1/yellow_tripdata_2023-02.parquet')

# 2. Compute the duration in minutes
data_feb['duration'] = (data_feb['tpep_dropoff_datetime'] - data_feb['tpep_pickup_datetime']).dt.total_seconds() / 60

# 3. Filter durations between 1 and 60 minutes (inclusive)
filtered_feb = data_feb[(data_feb['duration'] >= 1) & (data_feb['duration'] <= 60)]

# 4. Prepare the feature matrix using the same DictVectorizer
categorical_feb = filtered_feb[['PULocationID', 'DOLocationID']].astype(str)
dicts_feb = categorical_feb.to_dict(orient='records')
X_val = dv.transform(dicts_feb)  # Use the same dv as before

# 5. Prepare the target variable
y_val = filtered_feb['duration'].values

# 6. Predict and calculate RMSE
y_pred_val = lr.predict(X_val)
rmse_val = np.sqrt(mean_squared_error(y_val, y_pred_val))
rmse_val


np.float64(7.811817544074326)